In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
import re
import yaml

In [3]:
soccernet_tracking_set = "/kaggle/input/soccernet-tracking"

In [4]:
training_dirs = sorted(glob("/kaggle/input/soccernet-tracking/train/SNMOT*"))

val_dirs = training_dirs[:5]
training_dirs = training_dirs[:5]

split_dirs = {
    'train':training_dirs,
    'validate' : val_dirs
}

labels = ['ball','player','referee','goalkeeper']
label_dict = {'ball':0,'player':1,'referee':2,'goalkeeper':3,'goalkeepers':3}

In [7]:
print(val_dirs)

['/kaggle/input/soccernet-tracking/train/SNMOT-060', '/kaggle/input/soccernet-tracking/train/SNMOT-061', '/kaggle/input/soccernet-tracking/train/SNMOT-062', '/kaggle/input/soccernet-tracking/train/SNMOT-063', '/kaggle/input/soccernet-tracking/train/SNMOT-064']


In [9]:
DATA_DIR = "/kaggle/working/data"

In [5]:
yolo_train_img_dir = f"{DATA_DIR}/images/train"
yolo_val_img_dir = f"{DATA_DIR}/images/val"

yolo_train_label_dir = f"{DATA_DIR}/labels/train"
yolo_val_label_dir = f"{DATA_DIR}/labels/train"

yaml_file = f'{DATA_DIR}/data.yaml'

os.makedirs(yolo_train_img_dir,exist_ok=True)
os.makedirs(yolo_val_img_dir,exist_ok=True)
os.makedirs(yolo_train_label_dir,exist_ok=True)
os.makedirs(yolo_val_label_dir,exist_ok=True)



In [6]:
def get_yolo_format_bbox(img_w, img_h, box):
    w = box[2]
    h = box[3]
    xc = box[0] + int(np.round(w/2))
    yc = box[1] + int(np.round(h/2))
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box

In [7]:
def get_info(info):
    results = []
    for line in open(info):
        m = re.match('trackletID_(\d+)= (\S*).*', line.replace(';', ' '))
        if m:
            if m.group(2) not in label_dict:
                continue 
            results.append([m.group(1), m.group(2)])
    return pd.DataFrame(results, columns=['id','lbl']).set_index('id').to_dict()['lbl']

In [8]:
for split in split_dirs:
    if split == 'train':
        yolo_img_dir = yolo_train_img_dir
        yolo_label_dir = yolo_train_label_dir
    else:
        yolo_img_dir = yolo_val_img_dir
        yolo_label_dir = yolo_val_label_dir
        
    for this_dir in split_dirs[split]:
        print('this_dir:',this_dir)
        video = this_dir.split('/')[-1]
        info = this_dir + '/gameinfo.ini'
        det = this_dir + '/gt/gt.txt'
        info_dict = get_info(info)
        det_df = pd.read_csv(det, names=['frame','player','x','y','w','h','f1','f2','f3','f4'], usecols=['frame','player','x','y','w','h'])
        det_df['label'] = det_df.player.astype(str).map(info_dict)
        det_df['label_id'] = det_df['label'].map(label_dict)
        ng_list = list(det_df[det_df.label_id.isnull()].label.unique())
        if len(ng_list)>0:
            det_df = det_df.dropna()
        for grp, grp_df in det_df.groupby('frame'):
            frame = f'{grp:06}'
            img_file = f'{this_dir}/img1/{frame}.jpg'
            dst_file = f'{yolo_img_dir}/{video}_{frame}.jpg'
            if not os.path.exists(dst_file):
                os.symlink(img_file, dst_file)
            img = cv2.imread(dst_file)
            height, width, _ = img.shape 
            bboxes = []
            for arr in grp_df[['x', 'y', 'w', 'h', 'label_id']].values:
                box = get_yolo_format_bbox(width, height, arr[:4])
                bboxes.append([arr[4]]+box)
            file_name = f'{yolo_label_dir}/{video}_{frame}.txt'
            with open(file_name, 'w') as f:
                for i, bbox in enumerate(bboxes):
                    bbox = [str(i) for i in bbox]
                    str_bbox = ' '.join(bbox)
                    f.write(str_bbox)
                    f.write('\n')

this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-060
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-061
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-062
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-063
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-064
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-060
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-061
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-062
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-063
this_dir: /kaggle/input/soccernet-tracking/train/SNMOT-064


In [9]:
data_yaml = dict(
    train = yolo_train_img_dir,
    val = yolo_val_img_dir,
    nc = 4,
    names = labels
)

with open(yaml_file, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [10]:
!pip install ultralytics -q

In [11]:
!pip install wandb -q


In [12]:
import wandb

In [13]:
# secret api-key: c0630c2b69b9bc251f31fdcef5d386f633eca55c //please use responsibly

In [16]:

wandb.login(key="8e7633e856a55dfd5a8fd6b0e68ea0f9bf567a5d")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#  use the convention of {firstName}_{Model used}_{epochs}_{run Number} in experiment Name

In [17]:
!yolo task=detect mode=train model=yolov8n.pt data={yaml_file} imgsz=640 epochs=10 batch=32  name=Karim_smoke_test_run #name=experiment name

100%|███████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 111MB/s]
Ultralytics YOLOv8.2.83 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/data/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=Karim_smoke_test_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa